# Create Database
Pulls create-database.yaml directly from Github.  
This colab resides at: https://colab.research.google.com/drive/1Wm9Bvi9pC66xNtxKHfaJEeIYuXKpb1TA?usp=sharing   
TO DO: Add Supabase table insert

In [22]:
import yaml
import pandas as pd
import duckdb
import requests
import base64
import sqlalchemy
from sqlalchemy import create_engine, Table, MetaData # Table, MetaData were from ChatGPT

# User inputs for GitHub - These are optional. We will instead load from a local file.
saveToGitHub = False
github_token = 'your_github_personal_access_token'
repo_owner = 'your_github_username'
repo_name = 'your_github_repo'
commit_message = 'Add DuckDB database file'

file_name = 'US-2020-17schema.duckdb'

In [23]:
# Load the YAML configuration file
createDatabaseYaml = "https://raw.githubusercontent.com/ModelEarth/OpenFootprint/main/impacts/exiobase/US-source/create-database.yaml"
try:
    response = requests.get(createDatabaseYaml)
    response.raise_for_status()  # Ensure we raise an error for bad status codes
    config = yaml.safe_load(response.text)
except Exception as e:
    print(f"Failed to load the YAML file from URL: {e}")
    exit(1)

# TO DO: Display the yaml on the right side
# Like we do in "Run Models" colab at model.earth/RealityStream

In [24]:
# GitHub path to source files for Commodity.csv, etc.
sourcePath = "https://raw.githubusercontent.com/ModelEarth/OpenFootprint/main/impacts/exiobase/US-source/2020/"

# Function to convert column names to CamelCase without underscores
def to_camel_case(snake_str):
    components = snake_str.split('_')
    return components[0].capitalize() + ''.join(x.title() for x in components[1:])

# Create DuckDB connection
con = duckdb.connect(database='example.duckdb', read_only=False)

# Process each table in the YAML configuration
for table_name, table_config in config.items():
    csv_file = table_config['source']
    print(f"Load: " + sourcePath + csv_file)
    df = pd.read_csv(sourcePath + csv_file)

    columns_map = table_config.get('columns', {})
    omit_columns = table_config.get('omit', [])

    # Drop omitted columns
    df.drop(columns=omit_columns, errors='ignore', inplace=True)

    # Rename columns as per the YAML configuration
    df.rename(columns=columns_map, inplace=True)

    # Generate new column names for remaining columns
    new_columns = {}
    for col in df.columns:
        if col not in columns_map.values():
            new_columns[col] = to_camel_case(col)
    df.rename(columns=new_columns, inplace=True)

    # Exclude 'Year' column if not explicitly included in columns_map
    if 'Year' not in columns_map.values() and 'Year' in df.columns:
        df.drop(columns=['Year'], inplace=True)

    # Append "US" to the table name
    table_name = table_name + "US"

    # Insert/Update the data into the DuckDB database
    df.to_sql(table_name, con, if_exists='replace', index=False)

# Close the DuckDB connection
con.close()

# After the eight .csv files are loaded, this panel runs for at least 20 more minutes.
# The resulting 8.4MB US-2020-17schema.duckdb file reside to the left in the folder.


Load: https://raw.githubusercontent.com/ModelEarth/OpenFootprint/main/impacts/exiobase/US-source/2020/Commodity.csv


<ipython-input-24-5be1cbc90a5a>:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql(table_name, con, if_exists='replace', index=False)


Load: https://raw.githubusercontent.com/ModelEarth/OpenFootprint/main/impacts/exiobase/US-source/2020/US_summary_import_factors_exio_2020_17sch.csv


<ipython-input-24-5be1cbc90a5a>:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql(table_name, con, if_exists='replace', index=False)


Load: https://raw.githubusercontent.com/ModelEarth/OpenFootprint/main/impacts/exiobase/US-source/2020/US_summary_import_factors_exio_2020_17sch.csv


...

<ipython-input-24-5be1cbc90a5a>:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql(table_name, con, if_exists='replace', index=False)


Load: https://raw.githubusercontent.com/ModelEarth/OpenFootprint/main/impacts/exiobase/US-source/2020/country_contributions_by_sector_2020.csv


<ipython-input-24-5be1cbc90a5a>:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql(table_name, con, if_exists='replace', index=False)


Load: https://raw.githubusercontent.com/ModelEarth/OpenFootprint/main/impacts/exiobase/US-source/2020/multiplier_df_exio_2020_17sch.csv


<ipython-input-24-5be1cbc90a5a>:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql(table_name, con, if_exists='replace', index=False)


In [12]:
# The resulting US-2020-17schema.duckdb file was manually placed here:
# https://github.com/ModelEarth/OpenFootprint/tree/main/impacts/exiobase/US-source

# Push the DuckDB file to GitHub - Not yet implemented
if saveToGitHub == True:
    push_to_github(file_name, github_token, repo_owner, repo_name, commit_message, file_name)

# Function to push the DuckDB file to GitHub
def push_to_github(file_path, github_token, repo_owner, repo_name, commit_message, file_name):
    with open(file_path, 'rb') as f:
        content = f.read()
    content_base64 = base64.b64encode(content).decode('utf-8')

    url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/contents/{file_name}"
    headers = {
        'Authorization': f'token {github_token}',
        'Content-Type': 'application/json'
    }
    data = {
        'message': commit_message,
        'content': content_base64
    }
    response = requests.put(url, headers=headers, json=data)
    if response.status_code == 201:
        print('File successfully pushed to GitHub.')
    else:
        print(f'Failed to push file to GitHub: {response.json()}')